In [220]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine,text
import requests
import time
import re, unicodedata, html
from dotenv import load_dotenv
from urllib.parse import quote_plus
from bs4 import BeautifulSoup, Comment, Doctype
import re
import time



# Cargar variables de entorno (tu GOOGLE_API_KEY desde el archivo .env)
load_dotenv()

True

In [221]:
# Credenciales de la Base de Datos ANTIGUA (saingo_v2)
DB_OLD_USER = "root"
DB_OLD_PASS = ""
DB_OLD_HOST = "127.0.0.1"
DB_OLD_NAME = "sai_v2"

# Credenciales de la Base de Datos NUEVA (saingo_v5)
DB_NEW_USER = "root"
DB_NEW_PASS = ""
DB_NEW_HOST = "127.0.0.1"
DB_NEW_NAME = "sai_v5"


##id mapping pero no la usare



endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("OPENAI_API_KEY")


##algo asi sera solo de ejemplo

REPORTS_STATIC_CONTENT_STRUCTURE2 = {
    "en": {
        # Puedes añadir campos aquí si los necesitas, ej: "author_bio", "related_links_title"
        "main_banner_title": "News & Updates",
        "main_banner_subtitle": "Stay informed about our latest activities and impact.",
    },
    "es": {
        "main_banner_title": "Noticias y Actualizaciones",
        "main_banner_subtitle": "Mantente informado sobre nuestras últimas actividades e impacto.",
    }
}

title="locura"

REPORTS_STATIC_CONTENT_STRUCTURE = {
   
       "en":{
           "image_test":"https://definicion.de/wp-content/uploads/2012/01/imagen-vectorial.png",
           "h1_title":title,
           "h3_1":"Lorem ipsum dolorr","paragraph_1":"Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nis ut aliquip ex ea commodo consequat. Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan et iusto odio dignissim. Qui blandit praesent luptatum zzril delenit augue duis dolore te feugait nulla",
           "paragraph_2":"Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nis ut aliquip ex ea commodo consequat. Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan et iusto odio dignissim. Qui blandit praesent luptatum zzril delenit augue duis dolore te feugait nulla",
           "h3_2":"Lorem ipsum do",
           "paragraph_3":"Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nis ut aliquip ex ea commodo consequat. Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan et iusto odio dignissim. Qui blandit praesent luptatum zzril delenit augue duis dolore te feugait nulla",
           "donate_title":"Help people in need",
           "donate_text":"START DONATING TODAY, MAKE THE DIFFERENCE",
           "donate_button":""},
           "es":{"h1_title":"","posted_by":"","h3_1":"","paragraph_1":"","paragraph_2":"","h3_2":"","paragraph_3":"","donate_title":"",
                                     "donate_text":"",
                                     "donate_button":""}
}

# Convertimos la estructura a un string JSON una sola vez para reutilizarlo.









In [222]:
##buscare los datos en blog_post() de la vieja base de datos

def create_db_engine(user, password, host, dbname):
    connection_string = f"mysql+pymysql://{user}:{password}@{host}/{dbname}?charset=utf8mb4"
    return create_engine(connection_string)

def fetch_old_gallery_data():
    engine = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME) ##creo un motor de busqueda
    query = "SELECT id, title, excerpt, slug, body, seo_title, meta_description, tags, campaign_id FROM blog_posts" ## selecciona los titulos y id de blog posts

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df

def fetch_old_gallery_data2():
    engine = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME) ##creo un motor de busqueda
    query = "SELECT id,title FROM pages" ## selecciona los titulos y id de pages

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df


def fetch_new_gallery_data():
    engine = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME) ##creo un motor de busqueda
    query = "SELECT title FROM reports" ## selecciona los titulos y id de blog posts

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df




def get_post_translations(post_id):
    engine = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME) ##creo un motor de busqueda
    """Obtiene todas las traducciones para un ID de blog_post y las devuelve como un diccionario."""
    query = text("""
        SELECT column_name, value 
        FROM translations 
        WHERE table_name = 'blog_posts' AND foreign_key = :post_id AND locale = 'es'
    """)

    
    translations = {}
    print("entre")
    try:
        with engine.connect() as connection:
            result = connection.execute(query, {'post_id': post_id})
            
            for row in result:
                translations[row.column_name] = row.value
    except Exception as e:
        print(f"⚠️  No se pudieron obtener traducciones para el post ID {post_id}: {e}")
    return translations


#!def fetch_new_gallery_data():
    #engine = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME) ##creo un motor de busqueda
    #query = "SELECT id,title FROM blog_posts" ## selecciona las imagenes de pages, esto para buscar el titulo que corresponde al id nuevo

    #with engine.connect() as connection:
        #df = pd.read_sql(query, connection)
    #return df
def normalize(s: str) -> str:
   
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\xa0", " ")                 # NBSP -> espacio
    s = re.sub(r"[\u200B-\u200D\uFEFF]", "", s) # quita zero-width
    s = re.sub(r"\s+", " ", s).strip()
    return s



def simplify_html(html_content):
    """Simplifica el HTML eliminando scripts, estilos y atributos innecesarios."""
    if not html_content or pd.isna(html_content):
        return ""
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        for element in soup(['script', 'style', 'meta', 'link', 'head', 'noscript', 'iframe']):
            element.decompose()
        for tag in soup.find_all(True):
            allowed_attrs = ['href', 'src'] # Conservar solo href para links y src para imágenes
            attrs = {attr: tag.attrs[attr] for attr in allowed_attrs if attr in tag.attrs}
            tag.attrs = attrs
        return str(soup)
    except Exception as e:
        print(f"⚠️ Error simplificando HTML: {e}")
        return html_content[:8000]


def load_prompt_template(filepath="promptreport.txt"):
    """Carga la plantilla del prompt desde un archivo."""
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read()
def is_empty_p(p):
    # text with NBSP removed
    txt = p.get_text().replace('\xa0', '').strip()
    # consider <p><br></p> (and whitespace) as empty
    only_br = all(getattr(child, "name", None) in (None, "br") for child in p.children)
    return (not txt) and only_br

def clean_html(text):

    soup = BeautifulSoup(text, "html.parser")
    listimages=soup.find_all('img')
    listdiv=soup.find_all('div')
    listp=soup.find_all('p')
    acumulado2=""

    acumulado = []
    needle = "emanuel espin"  # comparación insensible a mayúsculas
    for p in listp:
        p_text = p.get_text(separator=" ", strip=True)
        if needle in p_text.lower():
            p.decompose()  # elimina por completo esa etiqueta <p> del HTML
            continue
        if p_text:
            acumulado.append(p_text)
            
    if acumulado:
        texto_acumulado = "<p>" + "</p>\n<p>".join(acumulado) + "</p>\r"
    else:
        texto_acumulado=""
        texto=""
        for d in listdiv:
            for em in d.find_all('em'):
                em.decompose()   # elimina por completo la etiqueta y su texto
        for div in listdiv:
            acumulado.append(div.get_text())
        if acumulado:
             texto_acumulado = "<p>" + "</p>\n<p>".join(acumulado) + "</p>\r"
             soup = BeautifulSoup(texto_acumulado, "html.parser")
             for p in soup.find_all('p'):
                txt = p.get_text().replace('\xa0', '').strip()  # quita NBSP y espacios
                if txt:  # tiene texto “visible”
                    acumulado2=acumulado2+"<p>"+txt+"</p>\n"         
     
        return acumulado2
    

    return texto_acumulado
                
                    

def clean_html_new(html,tit):
    soup = BeautifulSoup(html, 'html.parser')
    for cap in soup.select('div.caption'):
        cap.decompose()
    cont=0
    text_en=""
    title=""
    for tag in soup.find_all(True):
        own_text = "".join(tag.find_all(string=True, recursive=False)).strip().replace("\xa0", " ")
        if own_text:
            cont=cont+1
            #print(tag)
            norm=normalize(tit)
            if(own_text.lower()!="By Emanuel Espin |".lower() and own_text!="Project Manager" and own_text!=title and own_text!="By Jean Perez - Project Leader" and len(own_text)>10 and own_text!="By steven - Project leader" and own_text!=norm and normalize(own_text)!="By steven malca - President" and own_text.lower()!="By Steven Malca - President".lower() and own_text.lower()!="By Jimmy Pena - Project Leader".lower() and tag.name!="a" and tag.name!="em"):
                if not (cont==1 and tag.find("strong")):
                    text_en=text_en+"<p>"+own_text+"</p>\r\n"
                    
                   
                if tag.find("strong"):  # aquí comprobamos si hay un <strong> adentro
                    text_en += f"<p><strong>{own_text}</strong></p>\r\n"
                 


                    
                      

                else:
                    title=own_text 
    return text_en          
            

   

     
        
  
    
    
    


def convertir_json(db,dbnew,id):


    project_id_to_assign=122
    prompt_template = load_prompt_template()
    cont=0
    position=0

    for i in range(0,len(db)):
        num=db.iloc[i].get('id')
        if(num==id):
            report=db.iloc[i]
            position=i

          

    print(report)
    

    ##translations_list = [get_post_translations(id, db) for id in db['id']]
    ##db['translations_es'] = translations_list

    translations=get_post_translations(id)
    #text_editor_content_json = process_body_with_ai(
            #report.get('body'), 
            #translations.get('body', report.get('body')), # Fallback al inglés si no hay traducción del body
            #prompt_template
        #)
   
                  

    ##print(db.iloc[84].get('title'))
    ## "es": translations.get('slug', report.get('slug', ''))
    text_en_body = clean_html_new(report.get('body', ''),report.get('title', ''))
    text_es_body = clean_html_new(translations.get('body', report.get('body', '')),translations.get('title', report.get('title', '')))

    title_en = "<h1>"+report.get('title', '')+"</h1><br>\r\n"
    title_es = "<h1>"+translations.get('title', report.get('title', ''))+"</h1>\r\n"

    text_en = title_en+text_en_body
    text_es = title_es+text_es_body

    #print(title_en)
  
    title_en_text = report.get('title', '')

   
  
    title_es_text = translations.get('title', report.get('title', ''))
    # ⚠️ Esto MODIFICA el objeto global:
    REPORTS_STATIC_CONTENT_STRUCTURE['en']['h1_title'] = title_en_text
    REPORTS_STATIC_CONTENT_STRUCTURE['es']['h1_title'] = title_es_text
    REPORTS_STATIC_CONTENT_JSON = json.dumps(REPORTS_STATIC_CONTENT_STRUCTURE, ensure_ascii=False)

  

    


    meta_en = {
                "seo_title": report.get('seo_title'),
            "seo_description": report.get('meta_description'),
            "keywords": report.get('tags'),
            "og_title": report.get('seo_title'),
            "og_description": report.get('meta_description')
        }
    meta_es = {
            "seo_title": translations.get('seo_title'),
            "seo_description": translations.get('meta_description'),
            "keywords": translations.get('tags'),
            "og_title": translations.get('seo_title'),
            "og_description": translations.get('meta_description')
        }
    record = {
            "project_id": project_id_to_assign,
            "slug": json.dumps({
                "en": report.get('slug', ''),
                "es": translations.get('slug', report.get('slug', '')) 
                
            },ensure_ascii=False),
            "title": json.dumps({
                "en": report.get('title', ''), 
                "es": translations.get('title', report.get('title', ''))
            }, ensure_ascii=False),
            "excerpt": json.dumps({
                "en": report.get('excerpt', ''), 
                "es": translations.get('excerpt', report.get('excerpt', ''))
            },ensure_ascii=False),
            "content": REPORTS_STATIC_CONTENT_JSON, # Usamos la nueva plantilla,
            
            "text_editor_content": json.dumps({
                "en": text_en, 
                "es": text_es
            },ensure_ascii=False),
            "meta": json.dumps({"en": meta_en, "es": meta_es}, ensure_ascii=False),
            "status": "published",
            "creator_id": 1,
            "editor_id": 1,



    }

   

    return record






def subir(record):

    if not record:
        print("No hay registros para cargar.")
        return
    engine_new = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME)
    check_project_query = text("SELECT COUNT(*) FROM projects WHERE id = :id")
    insert_report_query = text("""
        INSERT INTO reports (
            project_id, slug, title, excerpt, content, text_editor_content, meta,
            status, creator_id, editor_id, published_at, updated_at
        ) VALUES (
            :project_id, :slug, :title, :excerpt, :content, :text_editor_content, :meta,
            :status, :creator_id, :editor_id, NOW(), NOW()
        )
                               """)
    try:
        
        with engine_new.begin() as conn:
    
            exists = conn.execute(
                text("SELECT 1 FROM projects WHERE id = :id LIMIT 1"),
                {"id": record["project_id"]}
            ).scalar()
            if not exists:
                raise ValueError(f"project_id {record['project_id']} no existe en projects")
            result = conn.execute(insert_report_query, record)
            print("ok")
           
            
          
    except Exception as e:
        print(f"❌ Error al cargar datos en 'reports': {e}")
   
def comprobar(title):
    db_new=fetch_new_gallery_data()
    proof=False
    tamano=len(db_new)

    for j in range(0,tamano):
        json=db_new.iloc[j]
        cadena=json.get("title").split('"')
        final=cadena[3]
        #vector=cadena.split(",")
        #conversion=vector[0][7:len(vector[0])-1]
        #final=conversion.strip('"')
        
        

        if(final==title):
            
            
            proof=True
            return proof
    return proof


    
 
db_old=fetch_old_gallery_data()
db_old2=fetch_old_gallery_data2()


#print(db_old.iloc[84,1])

#print(int(db_old.iloc[84,2]))



#print(db_old.iloc[83,0])

#print(db_old2.iloc[9,1]) ##encuentro la page donde montare esto
 
slug=db_old.iloc[82,3]
body=db_old.iloc[82,4]




excerpt=db_old.iloc[82,4]



#print(slug)

#print(excerpt)




html = body



# Parse HTML
soup = BeautifulSoup(html, "html.parser")

# Get plain text (BeautifulSoup automatically converts HTML entities like &nbsp;)
clean_text = soup.get_text()



#print(list[0].get_text()+"\r\n")

listimages=soup.find_all('img')

listdiv=soup.find_all('div')

for d in listdiv:
    for em in d.find_all('em'):
        em.decompose()   # elimina por completo la etiqueta y su texto

#print(listdiv[2].get_text())  
campaing_id=20

for i in range(0,87):
    if pd.notna(db_old.iloc[i].get('campaign_id')):
      
        if(int(db_old.iloc[i].get('campaign_id'))==campaing_id):
            if(comprobar(db_old.iloc[i].get('title'))==False):
                id=db_old.iloc[i].get('id')
                convertir_json(db_old,db_old2,id)
                print(id)
                subir(convertir_json(db_old,db_old2,id))














id                                                                 26
title                          Together we can make a huge difference
excerpt                        Together we can make a huge difference
slug                           together-we-can-make-a-huge-difference
body                <div class="progressReport">\n<div class="stam...
seo_title                      Together we can make a huge difference
meta_description               together-we-can-make-a-huge-difference
tags                         Juntos podemos hacer una gran diferencia
campaign_id                                                      20.0
Name: 20, dtype: object
entre
26
id                                                                 26
title                          Together we can make a huge difference
excerpt                        Together we can make a huge difference
slug                           together-we-can-make-a-huge-difference
body                <div class="progressReport">\n<div cl